In [1]:
'''
Where are our enhancers? De novo? Tag counts?
'''

'\nWhere are our enhancers? De novo? Tag counts?\n'

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

,PeakID (cmd=annotatePeaks.pl /data/mm10/CD4/ChIP/H3K4me2/AND-CD4TCell-ChIP-H3K4me2-All-Conditions-KAA-14-10-20/regions.txt mm10 -size given -d AND-CD4TCell-ChIP-H3K4me2-0_1uM-PCC-24h-Rep1-KAA-14-10-20 AND-CD4TCell-ChIP-H3K4me2-100uM-K99A-24h-Rep1-KAA-14-10-20 AND-CD4TCell-ChIP-H3K4me2-10uM-K99A-24h-Rep1-KAA-14-10-20 AND-CD4TCell-ChIP-H3K4me2-10uM-PCC-24h-Rep1-KAA-14-10-20 AND-CD4TCell-ChIP-H3K4me2-No-Peptide-24h-Rep1-KAA-14-10-20),Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,...,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type,0.1uM PCC,100uM K99A,10uM K99A,10uM PCC,No Peptide
0,chr14-6782,chr14,54216572,54236514,+,3339.0,132.460,Intergenic,Intergenic,27386,...,ENSMUSG00000022174,Dad1,AI323713,defender against cell death 1,protein-coding,3804.73,2789.86,3597.54,2997.39,3549.24
1,chr14-5093,chr14,54152959,54187935,+,2006.0,27.647,Intergenic,Intergenic,83482,...,ENSMUSG00000022174,Dad1,AI323713,defender against cell death 1,protein-coding,2244.43,1623.13,2153.69,1769.90,2271.04
2,chr9-7378,chr9,32696495,32723615,+,1621.8,51.833,"intron (NM_001038642, intron 1 of 6)","intron (NM_001038642, intron 1 of 6)",14013,...,ENSMUSG00000032035,Ets1,AI196000|AI448617|D230050P06|Ets-1|Tpl1,"E26 avian leukemia oncogene 1, 5' domain",protein-coding,1757.26,1328.48,1798.77,1332.62,1938.15
3,chr11-15413,chr11,52261839,52283403,+,1331.9,35.727,"intron (NM_009331, intron 2 of 9)",(TGGG)n|Simple_repeat|Simple_repeat,9950,...,ENSMUSG00000000782,Tcf7,AI465550|TCF-1|Tcf1,"transcription factor 7, T cell specific",protein-coding,1394.20,1081.90,1431.08,1016.65,1734.57
4,chr5-15815,chr5,124010148,124036996,+,1212.1,28.193,"intron (NM_177876, intron 1 of 3)","intron (NM_177876, intron 1 of 3)",8688,...,ENSMUSG00000066278,Vps37b,2300007F24Rik|AI415429|BC026744,vacuolar protein sorting 37B (yeast),protein-coding,1302.82,1044.57,1260.92,1021.85,1485.11


In [30]:
def get_pairwise(filename):
    # Pairwise overlaps
    # Set up a matrix to do an nxn iteration...
    min_tags = 40

    data = pd.io.parsers.read_csv('data/{}'.format(filename), 
                                        sep='\t', 
                                        header=0)

    data = data.fillna(0)

    # Rename our columns of interest
    labels = ['No Peptide', '10uM K99A', '100uM K99A', '0.1uM PCC', '10uM PCC']
    last_five_columns = [labels[3], labels[2], labels[1], labels[4], labels[0]]

    data.columns = data.columns[:-5].tolist() + last_five_columns

    pairwise = np.empty([len(labels), len(labels)])

    for i, label1 in enumerate(labels):
        #others = labels[:]
        #others.remove(label1)
        for j, label2 in enumerate(labels):
            if j < i: 
                # Only fill out half the triangle.
                val = None
            elif j == i:
                # Let's stick total peaks in the diagonal.
                #solo_mask = data.filter(items=others).max(axis=1) > min_tags
                val = sum(data[label1] > min_tags)
            else:
                # Get shared peaks for i and j
                shared_mask = (data[label1] > min_tags) & (data[label2] > min_tags)
                val = sum(shared_mask)

            pairwise[i:, j] = val

    df = pd.DataFrame(pairwise, columns=labels, index=labels)
    df = df.fillna('')
    
    return df

In [31]:
df = get_pairwise('union_peaks_five_conditions.txt')
df

,No Peptide,10uM K99A,100uM K99A,0.1uM PCC,10uM PCC
No Peptide,16012,14951,12487,14783,13105
10uM K99A,,15616,12549,14838,13192
100uM K99A,,,12695,12603,12247
0.1uM PCC,,,,15569,13281
10uM PCC,,,,,13444


In [33]:
df = get_pairwise('union_peaks_five_conditions_H3K27Ac.txt')
df

,No Peptide,10uM K99A,100uM K99A,0.1uM PCC,10uM PCC
No Peptide,6410,3591,5791,5988,5977
10uM K99A,,3671,3557,3594,3584
100uM K99A,,,8877,8699,8789
0.1uM PCC,,,,9867,9368
10uM PCC,,,,,10021
